In [1]:
# built-in
import os

# third-party
import pandas as pd
import pickle
from plotly import graph_objs as go

In [38]:
def get_performance_metrics(overview):
    '''
    Parameters
    ---------- 
    overview: pd.DataFrame
        Dataframe with the overview of the results for a specific participant activity and device
    
    Returns
    -------
    metrics: list
        List with the performance metrics [ratio, precision, recall]
    ''' 

    ratio = ((len(overview["TP_i"]) + len(overview["TP_e"])) + (len(overview["FP_i"]) + len(overview["FP_e"]))) / ((len(overview["TP_i"]) + len(overview["TP_e"])) + (len(overview["FN_i"]) + len(overview["FN_e"])))
    precision = (len(overview["TP_i"]) + len(overview["TP_e"])) / ((len(overview["TP_i"]) + len(overview["TP_e"])) + (len(overview["FP_i"]) + len(overview["FP_e"])))
    recall = (len(overview["TP_i"]) + len(overview["TP_e"])) / ((len(overview["TP_i"]) + len(overview["TP_e"])) + (len(overview["FN_i"]) + len(overview["FN_e"])))

    return [round(ratio, 2), round(precision, 2), round(recall, 2)]


def get_delays(overview, overview_BIOPAC, sampling_frequency=100):
    '''
    Parameters
    ---------- 
    overview: pd.DataFrame
        Dataframe with the overview of the results for a specific participant, activity and device
    overview_BIOPAC: pd.DataFrame
        Dataframe with the overview of the results for a specific participant and activity for the BIOPAC device
        
    Returns
    -------
    delays: list
        List with delay measures: ["mean absolute delay (std absolute delay)", delay adjusted to length of mean tB]. The absolute delay is in seconds
    ''' 

    delays = overview["delay_i"] + overview["delay_e"]
    delays = pd.Series(delays) * (1/sampling_frequency)

    adjusted_mean_delay = delays.mean() / pd.Series(overview_BIOPAC["tB (s)"]).mean()

    return [f"{delays.mean().round(2)} $\pm$ {delays.std().round(2)}", adjusted_mean_delay.round(2)]


def get_fr_detection_performance(overview, target):
    '''
    Parameters
    ---------- 
    overview: pd.DataFrame
        Dataframe with the overview of the results for a specific device and target
    target: str
        Target of the analysis. Can be "ID" or "Activity"
    
    Returns
    -------
    fr_detection: pd.DataFrame
        Dataframe with the performance metrics (Ratio, Precision, Recall, Mean absolute delay +/- SD and Adjusted delay), accroding to sensor and target
    
    ''' 

    fr_detection = pd.DataFrame(columns=[target, "Sensor", "Ratio", "Precision", "Recall", "Mean absolute delay $\pm$ SD", "Adjusted delay"])
    sensor = {"ScientISST": "MAGResp", "BITalino": "PZT"}
    

    for key in overview.keys():
        
        for device in ["ScientISST", "BITalino"]:
            new_entry = {}
            new_entry[target] = key
            new_entry["Sensor"] = sensor[device]
            new_entry["Ratio"], new_entry["Precision"], new_entry["Recall"]  = get_performance_metrics(overview[key][device])
            new_entry["Mean absolute delay $\pm$ SD"], new_entry["Adjusted delay"] = get_delays(overview[key][device], overview[key]["BIOPAC"])
            fr_detection.loc[len(fr_detection)] = new_entry

    return fr_detection

### Activity-specific only FR analysis

In [30]:
# Transform overview from participant specific into all participants
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_all_participants = {}
for id in overview.keys():
    for activity in overview[id].keys():

        if activity not in overview_all_participants.keys():
            overview_all_participants[activity] = {"ScientISST": {}, "BIOPAC": {}, "BITalino": {}}

        for metric in ["TP_i", "TP_e", "FP_i", "FP_e", "FN_i", "FN_e", "delay_i", "delay_e"]:
            overview_all_participants[activity]["ScientISST"][metric] = overview_all_participants[activity]["ScientISST"].get(metric, []) + overview[id][activity]["ScientISST"][metric]
            overview_all_participants[activity]["BITalino"][metric] = overview_all_participants[activity]["BITalino"].get(metric, []) + overview[id][activity]["BITalino"][metric]
        
        overview_all_participants[activity]["BIOPAC"]["tB (s)"] = overview_all_participants[activity]["BIOPAC"].get("tB (s)", []) + overview[id][activity]["BIOPAC"]["tB (s)"].tolist()

overview_all_participants.keys()
        

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR', 'SNBm', 'UALm', 'UARm'])

In [39]:
get_fr_detection_performance(overview_all_participants, target="Activity")

,Activity,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,SNB,MAGResp,1.02,0.97,0.99,0.07 $\pm$ 0.43,0.02
1,SNB,PZT,1.14,0.87,0.98,0.39 $\pm$ 0.58,0.09
2,SGB,MAGResp,1.00,0.94,0.94,0.23 $\pm$ 0.95,0.02
3,SGB,PZT,1.62,0.60,0.98,0.87 $\pm$ 1.68,0.08
4,MIXB,MAGResp,0.99,0.99,0.98,0.07 $\pm$ 0.4,0.01
5,MIXB,PZT,1.12,0.84,0.94,0.47 $\pm$ 0.86,0.09
6,STNB,MAGResp,1.05,0.95,0.99,0.08 $\pm$ 0.37,0.02
7,STNB,PZT,1.05,0.93,0.98,0.31 $\pm$ 0.86,0.07
8,MCH,MAGResp,1.17,0.85,0.99,0.11 $\pm$ 0.4,0.03
9,MCH,PZT,1.59,0.61,0.96,0.09 $\pm$ 0.69,0.02


### Participant-specific only FR analysis

In [35]:
# Transform overview from participant specific into all participants
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_all_activities = {}

for id in overview.keys():
    for activity in overview[id].keys():

        if id not in overview_all_activities.keys():
            overview_all_activities[id] = {"ScientISST": {}, "BIOPAC": {}, "BITalino": {}}

        for metric in ["TP_i", "TP_e", "FP_i", "FP_e", "FN_i", "FN_e", "delay_i", "delay_e"]:
            overview_all_activities[id]["ScientISST"][metric] = overview_all_activities[id]["ScientISST"].get(metric, []) + overview[id][activity]["ScientISST"][metric]
            overview_all_activities[id]["BITalino"][metric] = overview_all_activities[id]["BITalino"].get(metric, []) + overview[id][activity]["BITalino"][metric]
        
        overview_all_activities[id]["BIOPAC"]["tB (s)"] = overview_all_activities[id]["BIOPAC"].get("tB (s)", []) + overview[id][activity]["BIOPAC"]["tB (s)"].tolist()


overview_all_activities.keys()

dict_keys(['7OYX', 'NO15', 'G8B7', 'EPE2', 'HAK8', '1BST', '83J1', 'QMQ7', '9TUL', 'FTD7', 'Y6O3', '2QWT', 'F9AF', 'P4W9', 'W8Z9', 'D4GQ'])

In [40]:
get_fr_detection_performance(overview_all_activities, target="ID")

,ID,Sensor,Ratio,Precision,Recall,Mean absolute delay $\pm$ SD,Adjusted delay
0,7OYX,MAGResp,1.07,0.93,1.00,0.06 $\pm$ 0.38,0.01
1,7OYX,PZT,1.82,0.53,0.97,0.3 $\pm$ 0.91,0.06
2,NO15,MAGResp,1.17,0.84,0.99,0.05 $\pm$ 0.43,0.01
3,NO15,PZT,1.48,0.66,0.97,0.38 $\pm$ 0.77,0.09
4,G8B7,MAGResp,1.12,0.85,0.95,0.13 $\pm$ 0.56,0.04
5,G8B7,PZT,1.05,0.94,0.98,0.44 $\pm$ 0.45,0.14
6,EPE2,MAGResp,1.03,0.94,0.97,-0.18 $\pm$ 0.49,-0.04
7,EPE2,PZT,1.25,0.75,0.95,0.46 $\pm$ 0.81,0.10
8,HAK8,MAGResp,1.09,0.90,0.99,0.09 $\pm$ 0.34,0.03
9,HAK8,PZT,1.42,0.66,0.93,-0.09 $\pm$ 0.75,-0.03
